In [ ]:
# THIS CODE WAS PARTIALLY CREATED USING CHATGPT

In [2]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 11868    0 11868    0     0  59132      0 --:--:-- --:--:-- --:--:-- 59044
######################################################################## 100.0%#=#=#                                                                          
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!pip install ollama

In [9]:
import subprocess

In [13]:
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
subprocess.Popen(["ollama", "run" "llava:34b-v1.6-q2_K"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

<Popen: returncode: None args: ['ollama', 'runllava:34b-v1.6-q2_K']>

In [19]:
!ollama list

NAME               	ID          	SIZE 	MODIFIED      
llava:34b-v1.6-q2_K	6326f59da4f1	13 GB	6 minutes ago	


In [21]:
from PIL import Image

In [36]:
import os
from PIL import Image
from io import BytesIO
from ollama import generate, chat

# Function to read the image from a file
def read_image_file(file_path):
    with open(file_path, 'rb') as file:
        return file.read()

# Function to explain the image using the llava model
def explain_image(image_data):
    explanation = ""
    for response in generate('llava:34b-v1.6-q2_K', "You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. By the end of your investigation you have to have at least a rough guess of the image's location. Be creative and use deduction and logic to get to your final guess. MAKE ONE FINAL GUESS ABOUT THE IMAGE'S LOCATION, EVEN IF IT'S WRONG. DECIDE ON ONE FINAL DECISION, EVEN IF IT'S WRONG, JUST GUESS.", images=[image_data], stream=True):
        explanation += response['response']
    return explanation

# Function to chat with the model based on an initial message
def chat_with_model(initial_message):
    messages = [
        {
            'role': 'user',
            'content': initial_message,
        },
    ]

    response = chat('llava:34b-v1.6-q2_K', messages=messages)
    return response['message']['content']

# Main function to read an image, explain it, and chat about the explanation
def main(image_file_path):
    if not os.path.isfile(image_file_path):
        print("The file does not exist.")
        return

    image_data = read_image_file(image_file_path)
    
    explanation = explain_image(image_data)
    print("Image explanation:", explanation)
    print('---')

    chat_response = chat_with_model(explanation + """ \n Summarize the above in JSON as such: '"image_loc": "image location, country"'""")
    print("Chat response:", chat_response)


In [ ]:
main('/kaggle/input/llms-for-image-geolocation-benchmarks/im2gps3k_places365/im2gps3ktest/1000269685_e60e9cdfb4_1125_78841376@N00.jpg')


In [ ]:
1000269685_e60e9cdfb4_1125_78841376@N00.jpg